In [1]:
from matplotlib import pyplot as plt
import h5py
import numpy as np
import matplotlib.gridspec as gridspec
import random
from astroML.correlation import two_point
import stats
from tqdm import *
from scipy.spatial import distance
from scipy import spatial

cdm = h5py.File('Data/COLOR_CDM_DM_subHaloes_z0.00_v2.hdf5', 'r')
wdm = h5py.File('Data/COLOR_WDM_DM_subHaloes_z0.00_v2.hdf5', 'r')

cdm_galaxy = h5py.File('Data\COLOR_CDM_galaxies.hdf5', 'r')
wdm_galaxy = h5py.File('Data\COLOR_WDM_galaxies.hdf5', 'r')

print(cdm,wdm,cdm_galaxy,wdm_galaxy)

<HDF5 file "COLOR_CDM_DM_subHaloes_z0.00_v2.hdf5" (mode r)> <HDF5 file "COLOR_WDM_DM_subHaloes_z0.00_v2.hdf5" (mode r)> <HDF5 file "COLOR_CDM_galaxies.hdf5" (mode r)> <HDF5 file "COLOR_WDM_galaxies.hdf5" (mode r)>


# Access the masses and positions of CDM and WDM halos

In [2]:
# Access mass/position for wdm/cdm
print(list(cdm.keys()),list(wdm.keys()))

cdm_mass = cdm['SubhaloMass'][:]
cdm_position = cdm['SubhaloPos'][:]

wdm_mass = wdm['SubhaloMass'][:]
wdm_position = wdm['SubhaloPos'][:]

cdm_mass, cdm_position, wdm_mass, wdm_position

['IsCentral', 'R200', 'SubhaloMass', 'SubhaloPos'] ['IsCentral', 'R200', 'SubhaloMass', 'SubhaloPos']


(array([1.87617493e+14, 1.84468594e+14, 6.89978024e+13, ...,
        1.76011232e+08, 1.76011232e+08, 1.76011232e+08]),
 array([[10.93832207, 81.23078918, 54.77935028],
        [17.94562721, 79.86347961, 53.3843956 ],
        [15.62266159, 78.18474579, 52.83857346],
        ...,
        [ 1.47748184,  3.54455233, 99.53264618],
        [99.32190704,  2.56014371,  2.57368279],
        [99.0621109 ,  4.18381071,  2.89533401]]),
 array([1.89941540e+14, 1.86912732e+14, 6.83256610e+13, ...,
        1.76011232e+08, 1.76011232e+08, 1.76011232e+08]),
 array([[1.80150642e+01, 7.98655701e+01, 5.34328232e+01],
        [1.09048624e+01, 8.12610779e+01, 5.48179703e+01],
        [1.56323385e+01, 7.81665344e+01, 5.28157997e+01],
        ...,
        [6.02296066e+00, 5.05078554e+00, 9.79758301e+01],
        [5.75055540e-01, 3.26677370e+00, 3.52767438e-01],
        [2.07897183e-03, 3.23730040e+00, 9.97708917e-01]]))

# Access the masses and positions of CDM and WDM galaxies

In [3]:
# Access mass/position for wdm/cdm
print(list(cdm_galaxy.keys()),list(wdm_galaxy.keys()))

cdm_galaxy_color = cdm_galaxy['Colour'][:]
cdm_galaxy_position = cdm_galaxy['GalaxyPos'][:]
cdm_galaxy_mass = cdm_galaxy['StellarMass'][:]

wdm_galaxy_color = wdm_galaxy['Colour'][:]
wdm_galaxy_position = wdm_galaxy['GalaxyPos'][:]
wdm_galaxy_mass = wdm_galaxy['StellarMass'][:]

cdm_galaxy_color, cdm_galaxy_position, cdm_galaxy_mass, 
wdm_galaxy_color, wdm_galaxy_position,wdm_galaxy_mass

['Colour', 'GalaxyPos', 'IsCentral', 'StellarMass'] ['Colour', 'GalaxyPos', 'IsCentral', 'StellarMass']


(array([0.29584217, 0.29859257, 0.59355354, ..., 0.32156372, 0.59656096,
        0.32803154]), array([[17.61932182, 79.06235504, 52.68593979],
        [18.65647697, 80.58162689, 52.30108261],
        [18.58250046, 80.91363525, 52.45199203],
        ...,
        [76.68208313, 28.50509262, 59.96631622],
        [33.62237167,  9.74790382,  4.82571173],
        [52.58901978, 44.79496765, 78.47211456]]), array([3873670.  , 5328977.  , 1002360.25, ..., 2697151.5 ,  542400.5 ,
        2462471.  ]))

# Find central halos and central galaxies

In [4]:
# find the indices of all the central halos
cdm_central_ind = np.where(cdm['IsCentral'][:] == 1)[0]

print(cdm_central_ind)
print(len(cdm_mass[cdm_central_ind]))
print(cdm['IsCentral'][:][cdm_central_ind])

wdm_central_ind = np.where(wdm['IsCentral'][:] == 1)[0]

[      0   20093   40287 ... 4770038 4770039 4770040]
3961192
[1 1 1 ... 1 1 1]


In [5]:
cdm_galaxy_central_ind=np.where(cdm_galaxy['IsCentral'][:]==1)[0]
wdm_galaxy_central_ind=np.where(wdm_galaxy['IsCentral'][:]==1)[0]
print(cdm_galaxy['IsCentral'][:][cdm_galaxy_central_ind])

[1 1 1 ... 1 1 1]


# Import the sphere and envelope masses

In [6]:
cdm_5Mpc_sphere_mass = np.genfromtxt("cdm_5Mpc_sphere_mass.txt")
wdm_5Mpc_sphere_mass = np.genfromtxt("wdm_5Mpc_sphere_mass.txt")

In [7]:
cdm_sphere_mass=cdm_5Mpc_sphere_mass-cdm_mass[cdm_central_ind]
wdm_sphere_mass=wdm_5Mpc_sphere_mass-wdm_mass[wdm_central_ind]
len(cdm_sphere_mass), len(cdm_central_ind),len(wdm_sphere_mass),len(wdm_central_ind)

(3961192, 3961192, 2609122, 2609122)

In [8]:
cdm_10Mpc_sphere_mass = np.genfromtxt("cdm_10Mpc_sphere_mass.txt")
wdm_10Mpc_sphere_mass = np.genfromtxt("wdm_10Mpc_sphere_mass.txt")

In [9]:
cdm_envelope_mass=cdm_10Mpc_sphere_mass-cdm_5Mpc_sphere_mass
wdm_envelope_mass=wdm_10Mpc_sphere_mass-wdm_5Mpc_sphere_mass
len(cdm_envelope_mass),len(wdm_envelope_mass)

(3961192, 2609122)

# Match the galaxy indices to the right halos

In [10]:
len(cdm_central_ind),len(cdm_galaxy_central_ind)

(3961192, 181751)

In [11]:
cdm_galaxy['IsCentral'][:][0]

1

In [12]:
cdm_position_central = cdm_position[cdm_central_ind]
wdm_position_central = wdm_position[wdm_central_ind]

In [13]:
gal_now=cdm_galaxy_position[cdm_galaxy_central_ind]
halo_now=cdm_position_central[0]

dx=gal_now[:,0]-halo_now[0]
dy=gal_now[:,1]-halo_now[1]
dz=gal_now[:,2]-halo_now[2]

r = np.sqrt( dx**2 + dy**2 + dz**2)
print(np.argmin(r),r,np.min(r))

150504 [  6.61166377   8.23489217  13.83543203 ... 102.68923204  61.53852688
   7.22424264] 0.0


In [14]:
wdm_position[0], np.argmax(cdm_galaxy_mass)

(array([18.01506424, 79.86557007, 53.43282318]), 369865)

### Jk this is stupid

In [15]:
def distance(halo_list,galaxy):
    distance = []
    for i in trange(len(halo_list)):
        d=0
        for j in range(3):
            d += (halo_list[i][j]-galaxy[j])**2
        distance.append(d)
    return distance

In [16]:
halo_list=[[2,0,0],[0,1,0]]
print(halo_list[0])
dist=distance(halo_list,[0,0,1])
print(dist)
print(np.argmin(dist),halo_list[np.argmin(dist)])

[2, 0, 0]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


[5, 2]
1 [0, 1, 0]


In [38]:
# cdm_galaxy_iscentral=np.where(cdm_galaxy_central==1)[0]

# diff=distance(cdm_position[cdm_central_ind],cdm_galaxy_position[0])
# # print(diff)
# print(np.argmin(diff),cdm_central_ind,cdm_central_ind[np.argmin(diff)])
# cdm_galaxy_position[0]

# cdm_halo_match=[]
# for i in cdm_galaxy_iscentral:
#     diff=distance(cdm_position[cdm_central_ind],cdm_galaxy_position[i])
#     cdm_halo_match.append(cdm_central_ind[np.argmin(diff)])

cdm_halo_match=[]
for i in cdm_galaxy_central_ind:
    diff=distance(cdm_position[cdm_central_ind],cdm_galaxy_position[i])
    cdm_halo_match.append(cdm_central_ind[np.argmin(diff)])
    
print(cdm_halo_match)

In [17]:
# set(cdm_galaxy_mass[cdm_galaxy_iscentral]).intersection(cdm_mass[cdm_central_ind])
set(cdm_galaxy_mass[cdm_galaxy_central_ind]).intersection(cdm_mass[cdm_central_ind])

{13420856320.0, 14573730816.0}

# Let's do it the smart way: trees!

In [18]:
cdm_tree = spatial.cKDTree(cdm_position_central)
wdm_tree = spatial.cKDTree(wdm_position_central)

In [19]:
cdm_galaxy_position_central=cdm_galaxy_position[cdm_galaxy_central_ind]
wdm_galaxy_position_central=wdm_galaxy_position[wdm_galaxy_central_ind]

cdm_halo_match=cdm_tree.query(cdm_galaxy_position_central)
wdm_halo_match=wdm_tree.query(wdm_galaxy_position_central)

### try flipping it

In [20]:
cdm_tree = spatial.cKDTree(cdm_galaxy_position_central)
wdm_tree = spatial.cKDTree(wdm_galaxy_position_central)

cdm_halo_match=cdm_tree.query(cdm_position_central)
wdm_halo_match=wdm_tree.query(wdm_position_central)

In [21]:
print(cdm_halo_match,len(np.where(cdm_halo_match[0]>0)[0]))

(array([0.00000000e+00, 5.29834535e-04, 0.00000000e+00, ...,
       4.06106827e-01, 8.36804154e-01, 2.14681944e-01]), array([150504, 158855,  16902, ...,  38779, 123140, 118883])) 3802213


In [22]:
wdm_halo_match[0],cdm_tree.data, cdm_position_central
len(np.where(cdm_halo_match[0]>0)[0]),cdm_halo_match

(3802213, (array([0.00000000e+00, 5.29834535e-04, 0.00000000e+00, ...,
         4.06106827e-01, 8.36804154e-01, 2.14681944e-01]),
  array([150504, 158855,  16902, ...,  38779, 123140, 118883])))

In [23]:
print(len(cdm_tree.data),len(cdm_position),len(cdm_position_central))
print(cdm_halo_match[1])
print(cdm_mass[cdm_central_ind][cdm_halo_match[1]])
cdm_mass[cdm_central_ind][cdm_halo_match[1]]

181751 4770041 3961192
[150504 158855  16902 ...  38779 123140 118883]
[7.00524749e+09 6.53881754e+09 7.46639688e+10 ... 3.09603758e+10
 8.72135680e+09 9.04697754e+09]


array([7.00524749e+09, 6.53881754e+09, 7.46639688e+10, ...,
       3.09603758e+10, 8.72135680e+09, 9.04697754e+09])

In [24]:
print(len(cdm_position[cdm_central_ind][cdm_halo_match[1]]))
print()
print(len(cdm_position[cdm_halo_match[1]]))
#so they're different! --> cdm_halo_match corresponds to the indices of cdm_central_ind... right?

3961192

3961192


### Check that all the central galaxies match exactly to one central halo

In [30]:
print(cdm_halo_match,wdm_halo_match)
print(cdm_position[cdm_central_ind][cdm_halo_match[1][0]],cdm_galaxy_position[cdm_galaxy_central_ind[0]])
print(len(cdm_halo_match[1]),len(cdm_galaxy_central_ind),len(cdm_central_ind))

#success!
print(len(np.where(cdm_halo_match[0]!=0)[0]))
print(cdm['IsCentral'][:][cdm_central_ind][cdm_halo_match[1][np.where(cdm_halo_match[0]!=0)[0]]])
# print(np.where(cdm['IsCentral'][:][cdm['IsCentral'][:][np.where(cdm_halo_match[0]!=0)[0]]]==0)[0])
# len(np.where(cdm['IsCentral'][:][cdm_central_ind][cdm_halo_match[1]]==0)[0])
# print(cdm_halo_match[0])

(array([0., 0., 0., ..., 0., 0., 0.]), array([    274,     499,   20990, ..., 1655642, 1834262, 2745669])) (array([0.23595377, 0.21104215, 0.2602075 , ..., 0.        , 0.        ,
       0.        ]), array([704496, 355054, 141496, ..., 376772, 409690, 414022]))
[85.82241821 98.96107483 19.9845314 ] [17.43208122 80.25539398 54.00876999]
181751 181751 3961192
1812
[1 1 1 ... 1 1 1]


# Filter out central halos smaller than the resolution limit!

In [26]:
m_res = 8.8*10**6*50

In [27]:
cdm_halo_match_ind_res=np.where(cdm_mass[cdm_central_ind][cdm_halo_match[1]]>m_res)[0]
wdm_halo_match_ind_res=np.where(wdm_mass[wdm_central_ind][wdm_halo_match[1]]>m_res)[0]

In [28]:
print(cdm_halo_match_ind_res,cdm_halo_match)
print(len(cdm_halo_match_ind_res),len(cdm_halo_match[1]))
print(len(np.where(cdm_halo_match[0]!=0)[0]))

[      0       1       2 ... 3961189 3961190 3961191] (array([0.00000000e+00, 5.29834535e-04, 0.00000000e+00, ...,
       4.06106827e-01, 8.36804154e-01, 2.14681944e-01]), array([150504, 158855,  16902, ...,  38779, 123140, 118883]))
3959523 3961192
3802213


In [42]:
cdm_galaxy_position[cdm_galaxy_central_ind][cdm_halo_match_ind_res]

### Start with CDM

In [30]:
print(len(cdm_halo_match[1]),len(cdm_galaxy_position_central),
len(cdm_galaxy_central_ind),len(cdm_position[cdm_galaxy_central_ind]))

cdm_galaxy_central_ind_res=[]
for i in trange(len(cdm_galaxy_central_ind)):
    if cdm_mass[cdm_central_ind][cdm_halo_match[1][i]]>m_res:
        cdm_galaxy_central_ind_res.append(cdm_galaxy_central_ind[i])

3961192 181751 181751 181751


In [272]:
print(cdm_mass[cdm_central_ind][cdm_halo_match[1][0]]>m_res)
print(len(cdm_galaxy_central_ind_res),len(cdm_halo_match_ind_res))
print(m_res)

np.savetxt("cdm_galaxy_central_ind_res.txt", cdm_galaxy_central_ind_res, fmt = "%i")
cdm_galaxy_central_ind_res

True
180692 180692
440000000.0


0

### Repeat for WDM

In [32]:
wdm_galaxy_central_ind_res=[]
for i in trange(len(wdm_galaxy_central_ind)):
    if wdm_mass[wdm_central_ind][wdm_halo_match[1][i]]>m_res:
        wdm_galaxy_central_ind_res.append(wdm_galaxy_central_ind[i])

In [34]:
np.savetxt("wdm_galaxy_central_ind_res.txt", wdm_galaxy_central_ind_res, fmt = "%i")
wdm_galaxy_central_ind_res

# Now, search for surrounding galaxies within the virial radius

In [35]:
cdm_tree = spatial.cKDTree(cdm_galaxy_position)
wdm_tree = spatial.cKDTree(wdm_galaxy_position)

In [36]:
cdm['R200'][:],cdm['IsCentral'][:]

(array([ 1.12076938e+00, -9.99000000e+02, -9.99000000e+02, ...,
         1.13220578e-02,  9.71429888e-03,  1.09097715e-02]),
 array([1, 0, 0, ..., 1, 1, 1], dtype=int64))

### Filter out those below the resolution limit

In [37]:
cdm_central_ind_res=np.where(cdm_mass[cdm_central_ind]>m_res)[0]
len(cdm_central_ind),len(cdm_central_ind_res),len(cdm_halo_match_ind_res),len(cdm_galaxy_central_ind)

(3961192, 1686837, 3959523, 181751)

In [38]:
# create an empty array for the mass of the spheres
cdm_galaxy_sphere_position = []

#iterate through each central galaxy
for i in trange(len(cdm_galaxy_central_ind_res)):
    #find the indices for all halos in a virial radius around each central galaxy
    ind = cdm_tree.query_ball_point(cdm_galaxy_position[cdm_galaxy_central_ind_res[i]], 
                                        cdm['R200'][:][cdm_galaxy_central_ind_res][cdm_halo_match_ind[i]])
    #sum the mass of these halos
    cdm_galaxy_sphere_position.append(cdm_galaxy_position[ind])

In [39]:
#the indices are in the same order as cdm_galaxy_iscentral, which is in the same order as cdm_halo_match
cdm_galaxy_sphere_position=np.array(cdm_galaxy_sphere_position)
np.savetxt("cdm_galaxy_sphere_position.txt", cdm_galaxy_sphere_position, fmt = "%i")

In [40]:
# create an empty array for the mass of the spheres
wdm_galaxy_sphere_position = []

#iterate through each central galaxy
for i in trange(len(wdm_galaxy_central_ind_res)):
    #find the indices for all halos in a virial radius around each central galaxy
    ind = wdm_tree.query_ball_point(wdm_galaxy_position[wdm_galaxy_central_ind_res[i]], 
                                        wdm['R200'][:][wdm_galaxy_central_ind_res][wdm_halo_match_ind[i]])
    #sum the mass of these halos
    wdm_galaxy_sphere_position.append(wdm_galaxy_position[ind])

In [41]:
wdm_galaxy_sphere_position=np.array(wdm_galaxy_sphere_position)
np.savetxt("wdm_galaxy_sphere_position.txt", wdm_galaxy_sphere_position, fmt = "%i")